## Import libraries:

In [1]:
from pacti.iocontract.utils import getVarlist
from PIL import Image
from pacti.terms.polyhedra.loaders import readContract, writeContract

### Simple Example with pedestrian 5 steps away

In [2]:
## System contract: P >= 0.99
def system_contract():
    system_contract = {
        "InputVars":["xe"],
        "OutputVars":["P"],
        "assumptions":[
            {"coefficients":{"xe":-1}, "constant": -1},
            {"coefficients":{"xe": 1}, "constant": 1}
        ],
        "guarantees":[
            {"coefficients":{"P":-1}, "constant":-1},
            {"coefficients":{"P":1}, "constant":0}
        ]
    }
    return system_contract

## Perception contract P = p**5 = 5p - 4
def perception_contract():
    mc = {"InputVars":["p"],
        "OutputVars":[
            "P"
        ],
        "assumptions":
        [
            {"coefficients":{"p":1},
            "constant":1},
            {"coefficients":{"p":-1},
            "constant":0}
        ],
        "guarantees":
        [ # P = 5p - 4
          {"coefficients":{"P":1, "p":-5},
          "constant":-4},
          {"coefficients":{"P":-1, "p":5},
          "constant":4}
        ]}
    return mc

In [3]:
c1 = readContract(system_contract())
c2 = readContract(perception_contract())

print("Contract1:\n" + str(c1))
print("Contract2:\n" + str(c2))

Contract1:
InVars: [<Var xe>]
OutVars:[<Var P>]
A: -1*xe <= -1, 1*xe <= 1
G: -1*P <= -1, 1*P <= 0
Contract2:
InVars: [<Var p>]
OutVars:[<Var P>]
A: 1*p <= 1, -1*p <= 0
G: 1*P + -5*p <= -4, -1*P + 5*p <= 4


In [1]:
TILESIZE = 50
ORIENTATIONS = {'n': 270, 'e': 0, 's': 90,'w':180, 'ne':315, 'nw':225, 'se':45, 'sw':135}
START_CROSSWALK = 1
END_CROSSWALK = 5
CROSSWALK_V = 2
CROSSWALK_LOCATIONS = dict()
for i, num in enumerate(range(2*START_CROSSWALK,2*(END_CROSSWALK+1))):
    CROSSWALK_LOCATIONS.update({i: (num/2, CROSSWALK_V)})

### Example with robot and pedestrian

In [ ]:
class Grid():
    def __init__(self):
        self.xc_init = 0
        self.vc_init = 0
        self.ped = 1
        self.N = 5
        self.Vmax = 2
        self.xped = 5
        self.main_dir = os.path.dirname(os.path.dirname(os.path.realpath("__file__")))
        self.car_fig = main_dir + '/pacti/examples/evaluating_perception/imglib/red_car.png'
        self.ped_fig = main_dir + '/pacti/examples/evaluating_perception/imglib/pedestrian_img.png'
    
    def dynamics(self):
        transitions = dict()
        for x in range(0, self.N):
            x_succ = []
            for v in range(0, self.Vmax):
                x_succ.append(x + v)
            transitions[x] = x_succ
        return transitions
    
    def plot_grid(self):
        size = [1, self.N]
        x_min = 0
        x_max = size[0] * TILESIZE
        y_min = 0
        y_max = size[1] * TILESIZE
        #x_min, x_max, y_min, y_max = get_map_corners(map)
        ax.axis('equal')
        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)

        # fill in the road regions
        road_tiles = []
        width_tiles = np.arange(0,size[0]+1)*TILESIZE
        lanes_tiles = np.arange(0,size[1]+1)*TILESIZE

        for i in np.arange(0,size[0]+1):
            for k in np.arange(0,size[1]+1):
                tile = patches.Rectangle((width_tiles[k],lanes_tiles[i]),TILESIZE,TILESIZE,linewidth=1,facecolor='k', alpha=0.4)
                road_tiles.append(tile)
        ax.add_collection(PatchCollection(road_tiles, match_original=True))

        # now add crosswalk on top
        crosswalk_tiles = []
        for item in CROSSWALK_LOCATIONS.keys():
            if item % 2 == 0:
                color = 'silver'
                alpha = 0.5
            else:
                color = 'k'
                alpha = 0.5
            width = CROSSWALK_LOCATIONS[item][1]*TILESIZE
            lanes = CROSSWALK_LOCATIONS[item][0]*TILESIZE
            tile = patches.Rectangle((width,lanes),TILESIZE/2,TILESIZE,linewidth=1,facecolor=color, alpha=alpha)
            crosswalk_tiles.append(tile)
        ax.add_collection(PatchCollection(crosswalk_tiles, match_original=True))
        plt.gca().invert_yaxis()
    
    
    def plot_car(self):
        name, y_tile, x_tile, orientation = car_data
        theta_d = ORIENTATIONS[orientation]
        x = (x_tile) * TILESIZE
        y = (y_tile) * TILESIZE
        car_fig = Image.open(car_figs[color])
        car_fig = car_fig.rotate(theta_d, expand=False)
        offset = 0.1
        ax.imshow(car_fig, zorder=1, interpolation='bilinear', extent=[x+2, x+TILESIZE-2, y+2, y+TILESIZE-2])

# Need to specify Markov Chain as a symbolic library
class MC():
    def __init__(self, states, transitions, goal):
        self.S = states
        self. T = transitions
        self.F = goal
        self.P = {t: 0 for t in self.T} # Initialize transition probabilities
        
    def read_probabilities(self, transitions, P):
        pass

class CM():
    def __init__(self, classes):
        self.classes = classes
        self.row_data_type = "predicted_class"
        self.col_data_type = "true_class"
        self.class_dict = {i: classes[i] for i in range(len(classes))}
        self.rev_class_dict = {v: k for k,v in self.class_dict.items()}
        self.cm = np.zeros((len(classes)))
    
    def set_cm(self, cm):
        self.cm = cm
    
    def get_tpr(self, class_i):
        class_ind = self.rev_class_dict[class_i]
        return self.cm[class_ind, class_ind]
    
    def get_fpr(self, class_i):
        class_ind = self.rev_class_dict[class_i]
        fpr = sum(self.cm[class_ind, :]) - self.cm[class_ind, class_ind]
        return fpr
    
    def get_fnr(self, class_i):
        class_ind = self.rev_class_dict[class_i]
        total_samples = self.get_total_samples(class_i)
        return total_samples - self.cm[class_ind, class_ind]
    
    def get_total_samples(self, class_i):
        class_ind = self.rev_class_dict[class_i]
        total_samples = sum(self.cm[:, class_ind])
        return total_samples 
    
    def get_normalized_metrics(self, class_i):
        total_samples = self.get_total_samples(class_i)
        tpr = self.get_tpr(class_i)
        fpr = self.get_fpr(class_i)
        fnr = self.get_fnr(class_i)
        return tpr/total_samples, fpr/total_samples, fnr/total_samples
    
# Define child classes
class Proposition_CM():
    pass

class Distance_CM():
    pass

In [ ]:
## Object detection contract:
# Inputs: classes
# Outputs: true positive and false positive rates for each class


## Controller:
# At each time step or overall?
# Option 1: Overall probability analysis (not per time step)
# Challenge: How to generate the nonlinear expression and then simplify it to a linear constraint?
# Need not consider evolution.
# P = c + c*p1 + c*p2 + ...+ c*pn --> try to find coefficients